In [ ]:
#@title
###########################################################################
#
#  Copyright 2021 Google Inc.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# This solution, including any related sample code or data, is made available
# on an “as is,” “as available,” and “with all faults” basis, solely for
# illustrative purposes, and without warranty or representation of any kind.
# This solution is experimental, unsupported and provided solely for your
# convenience. Your use of it is subject to your agreements with Google, as
# applicable, and may constitute a beta feature as defined under those
# agreements.  To the extent that you make any data available to Google in
# connection with your use of the solution, you represent and warrant that you
# have all necessary and appropriate rights, consents and permissions to permit
# Google to use and process that data.  By using any portion of this solution,
# you acknowledge, assume and accept all risks, known and unknown, associated
# with its usage, including with respect to your deployment of any portion of
# this solution in your systems, or usage in connection with your business,
# if at all.
###########################################################################

# Imports, Globals, and Helper Functions

In [ ]:
#@title
!pip install boruta | grep -v 'already satisfied'
!pip install relativeImp | grep -v 'already satisfied'
from relativeImp import relativeImp
from sklearn.ensemble import RandomForestRegressor
from google.colab import files
from scipy.optimize import minimize, Bounds, NonlinearConstraint
from google.cloud import bigquery
from google.cloud.bigquery import magics
from google.colab import auth
from sklearn import preprocessing
from IPython.display import Javascript
import os
import re
import time
import random
import io
import sys
import json
import pandas as pd
import numpy as np
import ipywidgets as widgets
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import gspread
import warnings
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

TARGET_VAR = None
TARGET_VARIABLE = "TARGET_VARIABLE"

def current_milli_time():
    return round(time.time() * 1000)
timer = {}
def startTimer(line):
  global timer
  if line in timer.keys():
    timer[line]['start'] = current_milli_time()
  else:
    timer[line] = {}
    timer[line]['total'] = 0
    timer[line]['start'] = current_milli_time()

def endTimer(line):
  global timer
  timer[line]['total'] = timer[line]['total'] + (current_milli_time() - timer[line]['start'])

def Transformation(data, x, L, P, D):
    x_orig = data
    x_0 = np.append(np.zeros(L-1), x_orig)
    weights = np.zeros(L)
    for l in range(L):
      weight = D**((l-P)**2.0)
      weights[L-1-l] = weight
    adstocked_x = []
    for i in range(L-1, len(x_0)):
      x_array = x_0[i-L+1:i+1]
      xi = sum(x_array * weights)/sum(weights)
      adstocked_x.append(xi)
    return adstocked_x

def createTransformations(df1, df2, categoryMapping):
  columns = df2.columns
  sales = df1[[TARGET_VAR]]
  sys.setrecursionlimit(len(sales.index)+100)

  all_data = []
  for col in columns:
    if any(char.isdigit() for char in col):
      newdf = Transformation(df2, col, parseL(categoryMapping[stripQualifiers(col, False)]['fullColumn']), parseP(categoryMapping[stripQualifiers(col, False)]['fullColumn']), parseD(categoryMapping[stripQualifiers(col, False)]['fullColumn']))
      corr_df = pd.concat([sales, newdf], axis=1)
      corr = corr_df.corr().sort_values(TARGET_VAR, ascending=False)
      new_vals= corr.iloc[1:4 , 0:1].index.tolist()
      data = newdf[new_vals]
    else:
      data = df2[col]
    all_data.append(data)
  final_data = pd.concat(all_data,axis=1)

  return final_data

def parseL(column):
  reverse = column[::-1]
  l_value = reverse[reverse.find('p')+1 : reverse.find('l')][::-1].replace('_', '.')
  try:
    return float(l_value)
  except:
    return 8.0

def parseP(column):
  reverse = column[::-1]
  p_value = reverse[reverse.find('d')+1 : reverse.find('p')][::-1].replace('_', '.')
  return float(p_value)

def parseD(column):
  reverse = column[::-1]
  d_value = reverse[: reverse.find('d')][::-1].replace('_', '.')
  return float(d_value)

def stripQualifiers(column, leading=True):
  retString = column
  if retString.startswith('cost_') and leading:
    retString = retString.split('cost_')[1]
  if retString.startswith('Paid_') and leading:
    retString = retString.split('Paid_')[1]
  try:
    int(retString[::-1][retString[::-1].find('l')-1])
    retString = retString[::-1][retString[::-1].find('l')+1:][::-1]
    return retString
  except:
    return retString

final_iteration_revenue_data = []
def minimize_function(x, original_data, chosen_columns, pointEstimates):
  #print(original_data)
  final_revenue_data = []
  for attribute_idx, col in enumerate(chosen_columns):
    # idx is essentially the attribute we are using, order must be the same between x and columns of df

    #Step 1: Optimization
    #Opt.         = (original data X current opt spend) / sum original spend
    optimized_data = (original_data[stripQualifiers(col, False)] * x[attribute_idx]) / original_data[stripQualifiers(col, False)].sum()

    if any(char.isdigit() for char in col):
      with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        transformed_data = Transformation(optimized_data, stripQualifiers(col, False), int(parseL(col)), parseP(col), parseD(col))
    else:
      transformed_data = optimized_data

    #Step 2: Coeffs
    final_data = []
    #print(transformed_data)

    for i in transformed_data:
      final_data.append(i * pointEstimates[attribute_idx])

    #Step 3: Sum to revenue
    final_revenue_data.append(sum(final_data))


  #Set the final calculations for display later
  global final_iteration_revenue_data
  final_iteration_revenue_data = final_revenue_data

  return (sum(final_revenue_data) / sum(x)) * -1

def createBounds(mins, maxes):
  return Bounds(lb=mins, ub=maxes)

def constraintSum(x):
  return [sum(x)]

def createConstraints(budget):
  return NonlinearConstraint(fun=constraintSum, lb=[budget], ub=[budget])

def optimize(original_data, chosen_columns, pointEstimates, mins, maxes, budget):
  return minimize(fun=minimize_function, x0=mins, args=(original_data, chosen_columns, pointEstimates), method='SLSQP', bounds=createBounds(mins, maxes), constraints=createConstraints(budget))

# Import Data
Only one of the following sections should be executed, either import from CSV, BigQuery, or Google Sheets.

## Upload Original Data CSV (Option #1)

#### Run cell below first to select csv file.

In [ ]:
#@title
original_file = files.upload()
original_data_import = pd.read_csv(io.BytesIO(original_file[list(original_file.keys())[0]])).fillna(0)

## Import Original Data from BigQuery (Option #2)

In [ ]:
project_name = '' #@param {type:"string"}

In [ ]:
#@title
auth.authenticate_user()
bigquery.USE_LEGACY_SQL = False

magics.context.project = project_name
client = bigquery.Client(project=magics.context.project)
%load_ext google.cloud.bigquery
project_id =  project_name
dataset_name = '' #@param {type:"string"}
table_name = '' #@param {type:"string"}
query = 'select * from `' +  dataset_name + '.'+ table_name + '`'
original_data_import = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')
original_data_import.head()

## Import Data from Sheets (Option #3)

In [ ]:
auth.authenticate_user()
sheet_name = '' #@param {type:"string"}
worksheet = gc.open(sheet_name).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
original_data_import = pd.DataFrame.from_records(rows)
original_data_import = original_data_import.rename(columns=original_data_import.iloc[0]).drop(original_data_import.index[0]).reset_index(drop=True).astype('float')

# Import Relative Importance

Please include target variable & all features in final model. This will inform the feature level transformations for the optimization model.

Feature names should have L, P, and D adstock values.

#### Run cell below first to select csv file.

In [ ]:
#@title
print("Upload relative importance file")
importance_file = files.upload()
relativeImpDf = pd.read_csv(io.BytesIO(importance_file[list(importance_file.keys())[0]])).fillna(0)
relativeImpDf.head()

In [ ]:
feature_column_name = '' #@param {type:"string"}
attribution_column_name = '' #@param {type:"string"}


# Data Categorization
Run the cell below and then enter a comma separated list of channel names if you wish to group tactics by channel. [This is optional and can be left blank.]

i.e. Display, Social, YouTube...

**Do not re-run the sell after you input channel names**


In [ ]:
#@title
categoryList = widgets.Text(layout=widgets.Layout(width='500px'))
print("Comma separated list of optional channel names to organize your data")
categoryList

Now, run the cell below. Here you'll classify:

*   Target variable (in most cases this is your revenue column)
*   Non-paid media
*   Low funnel tactics
*   Channel (if channel was provided above)

**Note:** If channel names were not provided above and the feature is not the target, low funnel tactic, or non-paid media, you can leave blank.

**Do not re-run the sell after you classify the features**

In [ ]:
#@title
data_columns = original_data_import.columns.to_series()
categories = categoryList.value.split(',')
modelColumns = relativeImpDf[feature_column_name].copy()
categories[:] = [c.strip() for c in categories]
modelColumns[:] = [col.strip().replace('\'', '').replace('\"', '') for col in modelColumns]
categories.append("TARGET_VARIABLE")
categories.append("Non-Paid Media / Lower Funnel")
categoryDropdowns = []
categoryMapping = {}
for col in list(modelColumns):
  data_columns.drop(labels=[stripQualifiers(col, False)], inplace=True)
  dropdown = widgets.Dropdown(options=categories)
  categoryMapping[stripQualifiers(col, False)] = {'dropdown': dropdown, 'fullColumn': col}
  categoryDropdowns.append(widgets.HBox([widgets.Label(stripQualifiers(col), layout=widgets.Layout(width='300px')), dropdown]))
for col in list(data_columns):
  dropdown = widgets.Dropdown(options=categories)
  categoryMapping[col] = {'dropdown': dropdown, 'fullColumn': col}
  categoryDropdowns.append(widgets.HBox([widgets.Label(col, layout=widgets.Layout(width='300px')), dropdown]))
widgets.VBox(categoryDropdowns)

#Prep Data for Optimization
#### (Create Transformations, Standardization, Optimization Functions)

In [ ]:
#@title
targetVarsFound = 0
toBeDropped = []
for col in list(modelColumns) + list(data_columns):
  if categoryMapping[stripQualifiers(col, False)]['dropdown'].value == TARGET_VARIABLE:
    TARGET_VAR = col
    toBeDropped.append(TARGET_VAR)
    targetVarsFound += 1


if targetVarsFound == 1:
  #Variables that do not need to be transformed (i.e. lagged, carryover, diminishing returns)
  # Don't transform your y var, any flag variables, or variables that should be fully recognized on that specific day
  original_data_target_var = original_data_import[[
  TARGET_VAR,
  ]]

  #data that needs to be transformed
  trimmed_original_data = original_data_import.drop(toBeDropped, axis=1)

  #This step can take several minutes
  #This creates all combinations and then calculates the correlation between each variable and the Y variable.
  #Returns the top 3 highest correlated features

  transformed_original_data = createTransformations(original_data_target_var,
                                                    trimmed_original_data,
                                                    categoryMapping)
  feature_selected_transformed_w_target = pd.concat([original_data_target_var,
                                                     transformed_original_data],
                                                    axis=1)

  feature_selected_transformed_w_o_target = feature_selected_transformed_w_target.drop([TARGET_VAR], axis=1)

  #Calculate coefficients
  #Updated drive to feature
  impByDriver = relativeImpDf.set_index(feature_column_name)
  impByDriver['driverTrimmed'] = [stripQualifiers(i, False) for i in list(impByDriver.index.values)]
  impByDriver = impByDriver.set_index('driverTrimmed')
  impByDriver['revByTactic'] = impByDriver[attribution_column_name] * feature_selected_transformed_w_target[TARGET_VAR].sum()
  impByDriver['pointEstimate'] = np.zeros(len(impByDriver))
  for col in list(feature_selected_transformed_w_o_target):
    impByDriver['pointEstimate'][stripQualifiers(col, False)] = impByDriver['revByTactic'][stripQualifiers(col, False)] / original_data_import[stripQualifiers(col, False)].sum()
  impByDriver = impByDriver.sort_values(by=['pointEstimate'])
  recommendations = impByDriver.copy()
  dropRecommendations = []
  for col in list(feature_selected_transformed_w_target):
    if categoryMapping[stripQualifiers(col,False)]['dropdown'].value == "Non-Paid Media / Lower Funnel":
      dropRecommendations.append(stripQualifiers(col,False))
  recommendations = recommendations.drop(dropRecommendations)
  print("Recommendations:")
  if len(recommendations.index) > 5:
    print("Increase spend for: " + recommendations.index[-1] + ", " + recommendations.index[-2] + ", " + recommendations.index[-3])
  else:
    print("Increase spend for: " + recommendations.index[-1])

  if len(recommendations.index) > 3:
    if recommendations['pointEstimate'][recommendations.index[2]] < 1.0:
      print("Decrease spend for: " + recommendations.index[0] + ", " + recommendations.index[1] + ", " + recommendations.index[2])
    else:
      print("Decrease spend for: " + recommendations.index[0] + ", " + recommendations.index[1])
  else:
    print("Decrease spend for: " + recommendations.index[0])
else:
  print("Please ensure that one and only one TARGET_VARIABLE is chosen above.")



# **Optimizer RBA Tool**

## Instructions

Run the cell to dynamically generate the UI based on categorizations set above.

1.  Enter the total budget you wish to optimize.
2.  Enter min & max budget by tactic.
* Use all available information  to help inform the min & max (i.e. Last touch model, in platform attribution, ad hoc analyses, etc.)
* Pro-tip: press +/-20% to autofill min & max proportionally to original budgets
3.  Once you're comfortable with the min & max for each tactic, press "Do it!" to optimize budget.

## Additional Details

Based on the amount of data, the optimization may take a while. Interrupting the calculation may lead to an unresponsive runtime. If possible, please wait until the optimization finishes to make any changes.

## FAQ

What is the model optimizing for?
*  ROI. Baseline ROI is calculated with the original dataset. The optimization model is optimizing the overall budget in order to increase ROI.

What is the model optimizing ?
*  The model will optimize budgets for mid & upper funnel tactics

Confirm that this tool can not be used if impressions or clicks are modeled (unless backing into spend)
*  Correct, for this tool to work correctly, spend must be used for the tactic level input.



In [ ]:
#@title



#Layout and Styles for the buttons, inputs and loaders
buttonLayout = widgets.Layout(margin="10px 10px", width='300px')
pButtonLayout = widgets.Layout(margin="-25px 2px 10px 2px", width='300px')
button = widgets.Button(description="Do it!", layout=buttonLayout)
p20button = widgets.Button(description="+/- 20%", layout=pButtonLayout)
tableLayout = widgets.Layout(width='300px', display="flex", justify_content="center")
inputLayout = widgets.Layout(width='300px', display="flex", justify_content="center")
headerLayoutNoBottomMargin = widgets.Layout(width='300px', display="flex", justify_content="center")
headerLayout = widgets.Layout(width='300px', display="flex", justify_content="center", margin="0px 0px 50px")
warningLayout = widgets.Layout(width='300px', display="flex", justify_content="center")
html_styles = '''
    <style>
      .warning_1 input {
          color:red !important;
      }
      .warning_2 input {
          color:red !important;
      }
      .warning_3 input {
          color:red !important;
      }
      .warning_4 input {
          color:red !important;
      }
      .lds-dual-ring {
        display: inline-block;
        width: 30px;
        height: 30px;
        margin-bottom: 8px;
        margin-left: 20px;
        margin-top: 10px
      }
      .lds-dual-ring:after {
        content: " ";
        display: block;
        width: 16px;
        height: 16px;
        margin: 2px;
        border-radius: 50%;
        border: 3px solid #fff;
        border-color: #fff transparent #fff transparent;
        animation: lds-dual-ring 1.2s linear infinite;
      }
      @keyframes lds-dual-ring {
        0% {
          transform: rotate(0deg);
        }
        100% {
          transform: rotate(360deg);
        }
      }
      .widget-label {
        min-width:300px !important
      }
      .widget-text {
        min-width:300px !important
      }
      input {
        min-width:300px !important
      }
      .overflow_style {
        min-width:2400px !important
      }
    </style>
'''
html_loader = '''
  <div></div>
'''
rows = []
displayUI = []
metrics = {}

#Error handling is a little convoluted here, but it is easier to manage error classes across rows and columns when they map 1:1
errorMessage1 = "Attribute minimums exceed the category total minimum"
warning1 = "warning_1"
errorMessage2 = "Attribute maximums exceed the category total maximum"
warning2 = "warning_2"
errorMessage3 = "Attribute minimum exceeds attribute maximum"
warning3 = "warning_3"
errorMessage4 = "Category minimum exceeds category maximum"
warning4 = "warning_4"
errorMessage5 = "Consider increasing Search minimum to be within 20% of current budget"
warning5 = "warning_5"

#Number formatting to add comma every 3 digits
def formatNumber(number):
  return "{:,}".format(number)

def budgetChangeHandler(change):
  mins = []
  maxes = []
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        mins.append(attr['minSpend'].value)
        maxes.append(attr['maxSpend'].value)
  if metrics['budget'].value > sum(maxes) or metrics['budget'].value < sum(mins):
    printToOutputWidget(metrics['status'], "Your budget lies outside the min/max bounds for your table")
  else:
    printToOutputWidget(metrics['status'], "")

#Change handler fired every time input is changed. Handles error checking and category level updates
def attributeChangeHandler(change):
  for idx, attr in enumerate(rows):
    if attr['isCategoryRow']:
      attr['minSpend'].value = fetchTotalForCategory(attr['category'], True)
      attr['maxSpend'].value = fetchTotalForCategory(attr['category'], False)

    isMin = change.owner == attr['minSpend']
    if change.owner == attr['maxSpend'] or isMin:
      if fetchTotalForCategory(attr['category'], isMin) > fetchCategoryValue(attr['category'], isMin):
        if attr['isCategoryRow']:
          clearCategoryWarning(attr['category'], errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)
        setWarning(idx, errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)
      else:
        if attr['isCategoryRow']:
          clearWarning(idx, errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)
        clearCategoryWarning(attr['category'], errorMessage1 if isMin else errorMessage2, warning1 if isMin else warning2, isMin)

      if attr['minSpend'].value > attr['maxSpend'].value:
        if attr['isCategoryRow']:
          clearWarning(idx, errorMessage4, warning4, not isMin)
          setWarning(idx, errorMessage4, warning4, isMin)
        else:
          clearWarning(idx, errorMessage3, warning3, not isMin)
          setWarning(idx, errorMessage3, warning3, isMin)
      else:
        if attr['isCategoryRow']:
          clearWarning(idx, errorMessage4, warning4, isMin)
          clearWarning(idx, errorMessage4, warning4, not isMin)
        else:
          clearWarning(idx, errorMessage3, warning3, isMin)
          clearWarning(idx, errorMessage3, warning3, not isMin)

    if attr['category'] == 'Search' and not attr['isCategoryRow']:
      total = 0
      for idx2, attr2 in enumerate(rows):
        if not attr2['isCategoryRow'] and attr2['category'] != 'Total':
          total = total + original_data_import[attr2['attr']].sum()

      rowTotal = original_data_import[attr['attr']].sum()
      # ~78% to allow for rounding room in the initial 20% calculations
      p80 = (rowTotal/total) * metrics['budget'].value * 0.78
      if attr['minSpend'].value < p80:
        setWarning(idx, errorMessage5, warning5, isMin)
      else:
        clearWarning(idx, errorMessage5, warning5, isMin)

  mins = []
  maxes = []
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        mins.append(attr['minSpend'].value)
        maxes.append(attr['maxSpend'].value)
  if metrics['budget'].value > sum(maxes) or metrics['budget'].value < sum(mins):
    printToOutputWidget(metrics['status'], "Your budget lies outside the min/max bounds for your table")
  else:
    printToOutputWidget(metrics['status'], "")

#Helper function for error handling
def setWarning(rowIndex, message, warning, isMin):
  rows[rowIndex]['minSpend' if isMin else 'maxSpend'].add_class(warning)
  rows[rowIndex]['errors'].add(message)
  printToOutputWidget(rows[rowIndex]['warning'], message)

#Helper function for row level error handling
def clearWarning(rowIndex, message, warning, isMin):
  out = rows[rowIndex]['warning']
  rows[rowIndex]['minSpend' if isMin else 'maxSpend'].remove_class(warning)
  with out:
    if len(rows[rowIndex]['errors']) != 0:
      rows[rowIndex]['errors'].discard(message)
      out.clear_output()
      queueLength = len(rows[rowIndex]['errors'])
      if queueLength != 0:
        print(next(iter(rows[rowIndex]['errors'])))

#Helper function for category level error handling
def clearCategoryWarning(category, message, warning, isMin):
  for idx, attr in enumerate(rows):
    if attr['category'] == category:
      clearWarning(idx, message, warning, isMin)

#Getter function for category
def fetchCategoryValue(category, isMin):
  for idx, attr in enumerate(rows):
    if attr['category'] == category and attr['isCategoryRow']:
      return attr['minSpend' if isMin else 'maxSpend'].value

#Helper function to sum the values for a category
def fetchTotalForCategory(category, isMinTotal):
  total = 0
  for idx, attr in enumerate(rows):
    if attr['category'] == category and not attr['isCategoryRow']:
      total = total + (attr['minSpend'].value if isMinTotal else attr['maxSpend'].value)
  return total

#Generate a row for the output grid. Category rows have empty attribute values. This also populates
# the test data if desired.
def generateOptimizerRow(category, attr, pointEstimate, rows, isCategoryRow = False):
  categoryWidget = widgets.Label(category if isCategoryRow else "", layout=tableLayout)
  nameWidget = widgets.Label("" if isCategoryRow else stripQualifiers(attr), layout=tableLayout)
  min = 0
  max = 0
  minWidget = widgets.IntText(value=min, layout=inputLayout)
  maxWidget = widgets.IntText(value=max, layout=inputLayout)

  minWidget.observe(attributeChangeHandler, names='value')
  maxWidget.observe(attributeChangeHandler, names='value')

  optSpendWidget = widgets.Output(layout=tableLayout)
  optRevWidget = widgets.Output(layout=tableLayout)
  warningWidget = widgets.Output(layout=warningLayout)
  rows.append({
      'name': nameWidget,
      'minSpend': minWidget,
      'maxSpend': maxWidget,
      'optSpend': optSpendWidget,
      'optRev': optRevWidget,
      'warning': warningWidget,
      'category': category,
      'attr': attr,
      'pointEstimate': pointEstimate,
      'isCategoryRow': isCategoryRow,
      'errors': set(),
      'displayRow': widgets.HBox([categoryWidget, nameWidget, minWidget, maxWidget, optSpendWidget, optRevWidget, warningWidget])
       })

def runp20Click(b):
  total = 0
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow'] and attr['category'] != 'Total':
      total = total + original_data_import[attr['attr']].sum()

  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow'] and attr['category'] != 'Total':
      rowTotal = original_data_import[attr['attr']].sum()
      attr['maxSpend'].value = (rowTotal/total) * metrics['budget'].value * 1.2
      attr['minSpend'].value = (rowTotal/total) * metrics['budget'].value * 0.8
  return

#Execution function for the actual optimizer. Fires off the minimization functions and handles all metric output.
def runOptimizerClick(b):
  metrics['loader'].add_class('lds-dual-ring')
  mins = []
  maxes = []
  orderedPointEstimates = []
  chosen_columns = []
  optSpendTotal = 0
  optRevTotal = 0
  totalSpends = []
  global final_iteration_revenue_data
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        mins.append(attr['minSpend'].value)
        maxes.append(attr['maxSpend'].value)
        orderedPointEstimates.append(attr['pointEstimate'])
        for col in feature_selected_transformed_w_target:
          if stripQualifiers(col, False) == attr['attr']:
            chosen_columns.append(col)
        totalSpends.append(original_data_import[attr['attr']].sum())
      else:
        printToOutputWidget(attr['minSpend'], sum(mins))
        printToOutputWidget(attr['maxSpend'], sum(maxes))

  #Actual optimization run here
  res = optimize(original_data_import, chosen_columns, orderedPointEstimates, mins, maxes, metrics['budget'].value)
  roiOpt = res.fun*-1
  printToOutputWidget(metrics['roiWidget'], round(roiOpt, ndigits=5))

  #Metric calculations for pRoi, originalRoi
  calculate_proi_original_data = original_data_import.drop(columns=[col for col in original_data_import if col not in [stripQualifiers(i, False) for i in chosen_columns]])
  total_original_spend = calculate_proi_original_data.sum().to_frame().sum()
  total_original_revenue = 0
  for col in list(feature_selected_transformed_w_o_target):
    category = categoryMapping[stripQualifiers(col, False)]['dropdown'].value
    if category != TARGET_VARIABLE and category != 'IGNORE' and category != "Non-Paid Media / Lower Funnel":
      total_original_revenue = total_original_revenue + (impByDriver['pointEstimate'][stripQualifiers(col, False)] * feature_selected_transformed_w_o_target[col].sum())
  original_roi = total_original_revenue/total_original_spend
  printToOutputWidget(metrics['originalRoiWidget'], round(original_roi[0], ndigits=5))
  pRoi = ((roiOpt - original_roi)/original_roi) * 100
  printToOutputWidget(metrics['pRoiWidget'], str(round(pRoi[0], ndigits=2)) + '%')

  #Metric calculations for optimized spend, optimized revenue, and spend reallocated (not currently displayed)
  if(pRoi.any() > 0):
    printToOutputWidget(metrics['status'], res.message)
  else:
    printToOutputWidget(metrics['status'], "This optimization maximized ROI subject to the provided budget and tactic level constraints.")
  attributeIterator = 0
  optimizedRevenueBreakdown = []
  totalSpendReallocated = 0
  barChartInitialValues = totalSpends / sum(totalSpends)
  barChartOptimizedValues = []
  for idx, attr in enumerate(rows):
    if not attr['isCategoryRow']:
      if idx < len(rows)-1:
        barChartOptimizedValues.append(res.x[attributeIterator] / sum(res.x))

        printToOutputWidget(attr['optSpend'], round(res.x[attributeIterator], ndigits=2))
        printToOutputWidget(attr['optRev'], round(final_iteration_revenue_data[attributeIterator], ndigits=2))
        if not np.isclose([res.x[attributeIterator]], [0.0]):
          totalSpendReallocated = totalSpendReallocated + abs(res.x[attributeIterator]-totalSpends[attributeIterator])
        attributeIterator += 1
      else:
        printToOutputWidget(attr['optSpend'], round(sum(res.x), ndigits=2))
        printToOutputWidget(attr['optRev'], round(sum(final_iteration_revenue_data), ndigits=2))
        printToOutputWidget(metrics['spendReallocated'], round(totalSpendReallocated, ndigits=2))

  barWidth = 0.25
  fig = plt.subplots(figsize =(12, 8))

  # Set position of bar on X axis
  br1 = np.arange(len(barChartInitialValues))
  br2 = [x + barWidth for x in br1]
  br3 = [x + barWidth for x in br2]

  # Make the plot
  plt.bar(br1, barChartInitialValues * 100, color ='r', width = barWidth, edgecolor ='grey', label ='Initial')
  plt.bar(br2, np.array(barChartOptimizedValues) * 100, color ='g', width = barWidth, edgecolor ='grey', label ='Optimized')

  # Adding Xticks
  plt.xlabel('Tactic', fontweight ='bold', fontsize = 15)
  plt.ylabel('% of Overall Spend', fontweight ='bold', fontsize = 15)
  plt.xticks([r + barWidth for r in range(len(barChartOptimizedValues))],
          [stripQualifiers(x, True) for x in chosen_columns], fontsize=8, rotation=45)
  plt.legend()
  plt.show()
  metrics['loader'].remove_class('lds-dual-ring')

#Helper function for printing to an output widget
def printToOutputWidget(widget, data):
  with widget:
    widget.clear_output()
    if isinstance(data, str):
      print(data)
    else:
      print(formatNumber(data))

#Setup function for creating the display metrics in the grid
def createMetrics():
  budgetLabel = widgets.Label("Budget", layout=headerLayoutNoBottomMargin)
  roiLabel = widgets.Label("Optimized Return on " + stripQualifiers(TARGET_VAR, False), layout=headerLayoutNoBottomMargin)
  originalRoiLabel = widgets.Label("Original Return on " + stripQualifiers(TARGET_VAR, False), layout=headerLayoutNoBottomMargin)
  percentRoiLabel = widgets.Label("% Increase in Return on " + stripQualifiers(TARGET_VAR, False), layout=headerLayoutNoBottomMargin)
  spendReallocatedLabel = widgets.Label("Spend Reallocated", layout=headerLayoutNoBottomMargin)
  statusMessageLabel = widgets.Label("Status", layout=headerLayoutNoBottomMargin)
  budget = widgets.IntText(value=0, layout=headerLayout)
  budget.observe(budgetChangeHandler, names='value')
  originalRoiWidget = widgets.Output(layout=widgets.Layout(width='300px', display="flex", justify_content="center", margin="0px 0px 0px 8px"))
  roiWidget = widgets.Output(layout=headerLayout)
  pRoiWidget = widgets.Output(layout=headerLayout)
  spendReallocatedWidget = widgets.Output(layout=headerLayout)
  loaderWidget = widgets.HTML(html_loader)
  statusWidget = widgets.Output(layout=headerLayout)

  metrics= {
      'header': widgets.HBox([budgetLabel, originalRoiLabel, roiLabel, percentRoiLabel, statusMessageLabel]),
      'budget': budget,
      'originalRoiWidget': originalRoiWidget,
      'roiWidget': roiWidget,
      'pRoiWidget': pRoiWidget,
      'spendReallocated': spendReallocatedWidget,
      'loader': loaderWidget,
      'status': statusWidget,
      'displayRow': widgets.HBox([widgets.VBox([budget, p20button]), originalRoiWidget, roiWidget, pRoiWidget, statusWidget])
  }
  return metrics

#Setup function for creating the header labels
def createOptimizerHeader():
  nameLabel = widgets.Label("Attribute Name", layout=tableLayout)
  categoryLabel = widgets.Label("Category Name", layout=tableLayout)
  minLabel = widgets.Label("MIN Spend", layout=inputLayout)
  maxLabel = widgets.Label("MAX Spend", layout=inputLayout)
  optSpendLabel = widgets.Label("Optimized Spend", layout=tableLayout)
  optRevLabel = widgets.Label("Optimized " + stripQualifiers(TARGET_VAR, False), layout=tableLayout)
  warningLabel = widgets.Label("Warnings", layout=warningLayout)
  return widgets.HBox([categoryLabel, nameLabel, minLabel, maxLabel, optSpendLabel, optRevLabel, warningLabel])

#Setup function for creating the footers
def createOptimizerFooter():
  totalLabel = widgets.Label("Total", layout=tableLayout)
  spacerWidget = widgets.Label("", layout=tableLayout)
  minWidget = widgets.Output(layout=inputLayout)
  maxWidget = widgets.Output(layout=inputLayout)
  optSpendWidget = widgets.Output(layout=widgets.Layout(width='300px', display="flex", justify_content="center", margin="0px 0px 0px 8px"))
  optRevWidget = widgets.Output(layout=tableLayout)
  warningWidget = widgets.Output(layout=warningLayout)
  rows.append({
      'name': totalLabel,
      'minSpend': minWidget,
      'maxSpend': maxWidget,
      'optSpend': optSpendWidget,
      'optRev': optRevWidget,
      'isCategoryRow': False,
      'category': 'Total',
      'displayRow': widgets.HBox([totalLabel, spacerWidget, minWidget, maxWidget, optSpendWidget, optRevWidget, warningWidget])
       })

#Setup function for creating the optimizer grid
def populateRows():
  invertCategoryMapping = {}
  for col in categoryMapping.keys():
    if categoryMapping[col]['dropdown'].value not in invertCategoryMapping.keys():
      invertCategoryMapping[categoryMapping[col]['dropdown'].value] = [col]
    else:
      invertCategoryMapping[categoryMapping[col]['dropdown'].value].append(col)
  for category in invertCategoryMapping.keys():
    if category != TARGET_VARIABLE and category != 'IGNORE' and category != "Non-Paid Media / Lower Funnel":
      generateOptimizerRow(category = category, attr = "", pointEstimate = 0, rows = rows, isCategoryRow = True)
      for attr in invertCategoryMapping[category]:
        generateOptimizerRow(category = category, attr = attr, pointEstimate = impByDriver['pointEstimate'][attr], rows = rows)

#Setup function for coordinating the entire grid
def populateDisplayUI(displayUIList):
  global metrics
  metrics = createMetrics()
  #displayUIList.append(widgets.HTML(html_styles))
  displayUIList.append(metrics['header'])
  displayUIList.append(metrics['displayRow'])
  displayUIList.append(createOptimizerHeader())
  populateRows()
  createOptimizerFooter()
  for row in rows:
    displayUIList.append(row['displayRow'])
  button.on_click(runOptimizerClick)
  p20button.on_click(runp20Click)
  displayUIList.append(widgets.HBox([button, metrics['loader']]))

#Create grid and display it
populateDisplayUI(displayUI)
vbox = widgets.VBox(displayUI)
vbox.add_class('overflow_style')
widgets.VBox([widgets.HTML(html_styles), vbox])